# Scanning
- At this point you should have completed the reconnaissance stage of a penetration test and have collected information on the server/network as well as possible vulnerabilities
- The scanning portion of a penetration test will try to test the application for weaknesses and vulnerabilities to prepare for the next phase which is exploitation of weaknesses and vulnerabilities
- This section will cover procedures to detect common vulnerabilities and the tools that will help to discover them

### Using Hackbar for URL manipulation testing
- In OWASP Mantra, show hackbar using f9 or the green firefox icon on the left
- Hackbar will copy a url and allows you to change parameters
- Hackbar will not be affected by URL redirects and allows modification of POST parameters, SQL injection, and XSS

### Using Tamper Data to change submitted form data in-transit
- On OWASP Mantra use **Tools | Application Auditing | Tamper Data**
- Browse to the website you would like to use, and you can see the list populate with the address
- Intercept requests and change values with **Start Tamper**

### Using ZAP to view and alter requests
- This function, and the tool above are useful for getting around client-side filters when using forms
1. Open with `owasp-zap` command on CLI
2. Set up proxy server to be 127.0.0.1:8080 (Default ZAP port)
3. Enable **set break on all requests** on ZAP to begin request interception (Looks like a green "play" button on the top bar)
4. Submit a form on the webpage, then check the "Break" tab on ZAP to see the parameters being sent in the GET request
5. Change parameters as seen fit, then disable the **set break** button, then click the "play" button to continue with the form send 

**Note: BurpSuite can also be used in a very similar way**

### Finding XSS Vulnerabilities
- Anywhere that accepts user input and uses it as part of their html output or variables can be succeptible to XSS if they do not use input validation
- For example: HTML code that uses a script to output "Hello \<name\>" might be succeptible to this if someone were to add a script
- Basic test: `Bob<script>alert('XSS Test')</script>` would create an alert window with the text "XSS Test" if they did not use input validation

### Finding SQL Injection Vulnerabilities
- Any application which uses user input to query a database can be vulnerable to SQL injection if they do not do input validation
- Using an apostrophe `'` is the syntax used to close a string statement, and can be used as input to check for an error getting thrown that might indicate a SQLi vulnerable application
- If an error is thrown by that, but not by two apostrophes `''`, that is a good indicator of a possible SQLi vector
- Try a very basic SQL command with `' or '1'='1`, which will validate everything to true, possibly making all items in the database applicable for whatever query it is doing, outputting all possible values of that query

**Identifying Blind SQLi Vulnerablities**
- Some applications may be vulnerable but will not throw an error and are thus "Blind" vulnerabilities
- A good way to tell there is a Blind SQLi Vulnerability is to check for different output based on a `true` or `false` SQL statement
    1. Check for normal valid input, confirm a normal response
    2. Check for possible SQL injection with `'` and `''`
    3. Check for an always false input and response: `1' and '1'='2`
    4. Check for an always true input and response: `1' and '1'='1`
- If you get two different responses for a false SQLi and true SQLi, it is a strong indicator of a Blind SQLi

### Finding Cookie Vulnerabilities
- Many applications add cookies to the browser to keep track of information
- Check cookies with an application like Cookies Manager+
- Delete all cookies before visiting a website to see exactly which cookies are being added and when
- Choose a cookie and use the "Edit" function to see parameters
- `PHPSESSID` is the default name for session cookies using PHP
- Signs of a vulnerable cookie:
    1. The `HTTPOnly` flag is not enabled
        - This can indicate a XSS vulnerability that allows for session hijacking using the session ID
    2. The `Send For:` tag is not set to `Send for Encrypted Connections Only`
        - This means a MiTM attack can be done against a user using this cookie to receive session cookies via HTTP, which can be used for hijacking a session
- `ASP.NET_SessionID` - Naming convention used for ASP.NET session cookies
- `JSESSIONID` - Naming convention used for JSP session cookies

### Using SSLScan to obtain SSL and TLS information
- SSlScan is built into Kali and can be accessed with the `sslscan` command
- Scan an https website using the command `sslscan https://domain.com`
    - You can also use an https IP 
- The scan will reveal various vulnerabilities, specifications about services, encryption methods, ciphers, etc.
    - Items seen in **Red** means that they are now considered unsecure (e.g. RSA, DES)
    - Items seen in **Yellow** means that they are medium strength ciphers (e.g. RC4, SHA, MD5)
- This information is useful because a MiTM attack could go in knowing what ciphers are supported, allowing an attacker to trick the server into thinking the client wants to communicate using one of the weaker supported ciphers, which can be more easily deciphered by the attacker
- The SSLScan can also detect Heartbleed vulnerabilities which are a bug in OpenSSL which can be used to cause a buffer overflow on any OpenSSL version (1.0.1-1.0.1f)
    - This buffer overflow will read up to 64KB from server memory in plain text, repeatedly, and without leaving any trace or log on a server
    - An attacker can obtain plain text information such as the server private key or encryption certificates, user information, etc.
    
### Finding File Inclusion Vulnerabilities
- File Inclusion Vulnerability - Occurs when a developer uses request parameters from a user, which can then be modified to dynamically choose what pages to load or include in code that the server executes
- You may see this kind of vulnerability in a URL which may have a suffix of something like `somesubdirectory/whatever/?page=include.php`, with the `page=` being of interest
- Knowing what files a website uses is important, which is why web crawlers can be used to find the name of a file that really exists locally (e.g. `../../index.php` the index.php file)
1. Try directory traversal via `?page=../../index.php`
    - If this works it would mean that directory traversal and Local File Inclusion (LFI) is possible
2. Try remote file inclusion - Including a file hosted on another server instead of a local one
    - e.g. `?page=http://192.168.254.128/vicnum/index.html` (find a URL to insert that has a page to load)
    - If you are able to make the application load a page using the full URL, this means you can include remote files (RFI)
        - If this file contains server-side executable code (e.g. PHP), the code becomes executed by the local server, allowing an attacker to gain remote command execution, which is a full system compromise
        
**Why does this work?**
- By checking the View Source you may see something like:
```
<?php
$file = $_GET['page']; //Page being displayed
?>
```
- This means that the `page` variable being called is being passed directly as a filename included in the code, which means that you could include any PHP, HTML, etc. file into the server
- A server vulnerable to RFI must have `allow_url_fopen` and `allow_url_include` enabled in the configuration, otherwise only LFI vulnerabilities will be present
    - Obviously LFI is significantly less potent as you can only use files already on the local machine, but is still a vulnerability
        - We can use LFI to display relevant files in the host operating system
        - e.g. `../../../../../../../../../etc/passwd` (add enough ../ to navigate to the main directory) will get a list of system users, home directories, and default shells
       
### POODLE Vulnerability
- POODLE - Padding Oracle On Downgraded Legacy Encryption
    - Attack that uses a "Downgraded Legacy Encryption" to downgrade a TLS communication to SSLv3 (weaker encryption) and forces use of cipher suites (CBC), which can be easily broken and communications decrypted
- Testing for POODLE vulnerabilities with Nmap
    - \*Note: You must have the ssl-poodle.nse script from nmap.org/nsedoc/scripts
    - `nmap --script ssl-poodle -sV -p 443 192.168.254.128`
    - If it is vulnerable you will see a section of output like this:
        ```
        | ssl-poodle: 
        |   VULNERABLE:
        |   SSL POODLE information leak
        |     State: VULNERABLE
        ```
        
# Automated Scanning
- While manually trying to scan for vulnerabilities is always going to be necessary, one should always try to automate
- It is very useful for pen testing to have a tool which can perform a variety of tests on an application within a short period of time
- Here we will cover Kali based vulnerability scanners that target web application vulnerabilities commonly used by pen testers and cyber security professionals like ourselves

### Using Nikto Web Server Scanner
- Command line utility made to scan a host
- `nikto -h http://192.168.254.128/peruggia/ -o result.html`
    - This command calls nikto to scan a host (-h) and save the output (-o) as a file
    - The scan will take some time to complete, and once finished can be reviewed through the html file we saved

**Useful Flags**
- `-H or --help` : Shows Help page
- `-config <file>` : Uses a custom configuration file for the scan
- `-update` : Updates the database for Nikto
- `-Format <format>` : Defines the output format
- `-evasion <technique>`: Uses encoding technique to help avoid detection by a WAF or IDS/IPS
- `-Plugins <plugins>` : Select a plugin to use in a scan (default is ALL)
- `-list-plugins`: Shows a list of plugins that can be used with `-Plugins`
- `-port <port number>` : 